In [1]:
import numpy as np
from keras import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.layers import Input,Flatten,Dense,BatchNormalization,Activation,Dropout,GlobalAveragePooling2D,MaxPooling2D,RandomFlip,RandomZoom,RandomRotation
from keras.datasets import cifar10
import keras

C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [3]:
base_model = keras.applications.MobileNet(
    weights='imagenet',
    input_shape=(224,224, 3),
    include_top=False)

for layer in base_model.layers:
    layer.trainable = False

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Lambda
from tensorflow.keras.models import Model

inputs = tf.keras.Input(shape=(32, 32, 3))
x = Lambda(lambda image: tf.image.resize(image, (224, 224)))(inputs)
x = tf.keras.applications.xception.preprocess_input(x)
x = base_model(x, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
outputs = Dense(10, activation='softmax')(x)
model = Model(inputs, outputs)

In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
from keras.callbacks import EarlyStopping

epochs = 10
batch_size = 32

early_stopping = EarlyStopping(
    monitor='accuracy',
    patience = 1,
    restore_best_weights = True
)

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, callbacks=[early_stopping])

Epoch 1/10
1563/1563 [==============================] - 5015s 3s/step - loss: 0.6733 - accuracy: 0.7727
Epoch 2/10
1563/1563 [==============================] - 5254s 3s/step - loss: 0.5298 - accuracy: 0.8188
Epoch 3/10
1563/1563 [==============================] - 5329s 3s/step - loss: 0.5146 - accuracy: 0.8264
Epoch 4/10
1563/1563 [==============================] - 4615s 3s/step - loss: 0.5087 - accuracy: 0.8272
Epoch 5/10
1563/1563 [==============================] - 3050s 2s/step - loss: 0.5050 - accuracy: 0.8300
Epoch 6/10
1563/1563 [==============================] - 3025s 2s/step - loss: 0.5000 - accuracy: 0.8307
Epoch 7/10
1563/1563 [==============================] - 3040s 2s/step - loss: 0.5036 - accuracy: 0.8315
Epoch 8/10
1563/1563 [==============================] - 3009s 2s/step - loss: 0.5066 - accuracy: 0.8289


In [8]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')

313/313 - 535s - loss: 0.4562 - accuracy: 0.8456 - 535s/epoch - 2s/step

Test accuracy: 0.8456000089645386


In [9]:
model.save('NN/mobilenet_cifar10.h5')